In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pydicom

In [ ]:
# Makes it so that any changes in pymedphys is automatically
# propagated into the notebook without needing a kernel reset.
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
import pymedphys
from pymedphys._mosaiq.helpers import FIELD_TYPES

In [ ]:
data_paths = pymedphys.zip_data_paths("tel-dicom-pairs.zip")
dir_name = [path.parent.name for path in data_paths if 'BreastAndBoost' in path.parent.name][0]

dir_name

In [ ]:
def get_file_type(input_paths, file_type, exact_name=False):
    if exact_name:
        paths = [path for path in input_paths if path.name == file_type]
    else:
        paths = [path for path in input_paths if file_type in path.name]
        
    assert len(paths) == 1
    return paths[0]

In [ ]:
current_paths = [path for path in data_paths if path.parent.name == dir_name]
current_paths

In [ ]:
tel_path_fraction_1 = get_file_type(current_paths, "tel.1", exact_name=True)
tel_path_fraction_2 = get_file_type(current_paths, "rxB_tel.1", exact_name=True)
dcm_path = get_file_type(current_paths, "dcm")

In [ ]:
tel_path_fraction_1

In [ ]:
dcm_path

In [ ]:
delivery_dcm = pymedphys.Delivery.from_dicom(
    pydicom.read_file(str(dcm_path), force=True),
    fraction_number=1
)

delivery_monaco = pymedphys.Delivery.from_monaco(tel_path_fraction_1)

In [ ]:
plt.plot(delivery_monaco.gantry)
plt.plot(delivery_dcm.gantry)

In [ ]:
dcm_mudensity = delivery_dcm.mudensity()
monaco_mudensity = delivery_monaco.mudensity()

In [ ]:
GRID = pymedphys.mudensity.grid()

In [ ]:
pymedphys.mudensity.display(GRID, dcm_mudensity)

In [ ]:
pymedphys.mudensity.display(GRID, monaco_mudensity)

In [ ]:
diff = monaco_mudensity - dcm_mudensity

In [ ]:
pymedphys.mudensity.display(GRID, diff)

In [ ]:
def get_patient_fields(cursor, patient_id):
    """Returns all of the patient fields for a given Patient ID.
    """
    patient_id = str(patient_id)

    patient_field_results = pymedphys.mosaiq.execute(
        cursor,
        """
        SELECT
            TxField.FLD_ID,
            TxField.Field_Label,
            TxField.Field_Name,
            TxField.Version,
            TxField.Meterset,
            TxField.Type_Enum,
            Site.Site_Name
        FROM Ident, TxField, Site
        WHERE
            TxField.Pat_ID1 = Ident.Pat_ID1 AND
            TxField.SIT_Set_ID = Site.SIT_Set_ID AND
            Ident.IDA = %(patient_id)s
        """,
        {"patient_id": patient_id},
    )

    table = pd.DataFrame(
        data=patient_field_results,
        columns=[
            "field_id",
            "field_label",
            "field_name",
            "field_version",
            "monitor_units",
            "field_type",
            "site",
        ],
    )

    table.drop_duplicates(inplace=True)

    table["field_type"] = [FIELD_TYPES[item] for item in table["field_type"]]

    return table

In [ ]:
mosaiq_sql_host = 'physics-server:31433'
patient_id = '200054'

with pymedphys.mosaiq.connect(mosaiq_sql_host) as cursor:
    patient_fields = get_patient_fields(cursor, patient_id)
    
patient_fields

In [ ]:
with pymedphys.mosaiq.connect(mosaiq_sql_host) as cursor:
    mosaiq_delivery = pymedphys.Delivery.from_mosaiq(cursor, 7115)

In [ ]:
mosaiq_mudensity = mosaiq_delivery.mudensity()

In [ ]:
pymedphys.mudensity.display(GRID, mosaiq_mudensity)

In [ ]:
pymedphys.mudensity.display(GRID, mosaiq_mudensity - dcm_mudensity)

In [ ]:
# delivery_dcm.gantry

In [ ]:
len(delivery_monaco.gantry)

In [ ]:
len(delivery_dcm.gantry)

In [ ]:
len(delivery_monaco.mlc)

In [ ]:
len(delivery_dcm.mlc)

In [ ]:
monaco_mlc = np.array(delivery_monaco.mlc)
dcm_mlc = np.array(delivery_dcm.mlc)

In [ ]:
diff = monaco_mlc - dcm_mlc

In [ ]:
np.shape(diff)

In [ ]:
monaco_mlc[0,:,:]

In [ ]:
dcm_mlc[0,:,:]

In [ ]:
diff[0,:,:]

In [ ]:
np.where(np.abs(diff) > 0.09)